In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score

In [14]:
import numpy as np
class NaiveBayes:
    def fit(self, X, y):#X:data y:label
        n_samples, n_features = X.shape#number of row is n_samples, number of column is n_feature
        self._classes = np.unique(y)#_class conclude unquie of y
        n_classes = len(self._classes)

        # calculate mean, var, and prior for each class
        self._mean = np.zeros((n_classes, n_features), dtype=np.float64) #tao ma tran 0 2 chieu chua mean
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)#tao ma tran 0 2 chieu chua var
        self._priors = np.zeros(n_classes, dtype=np.float64)#tao ma tran 0 1 chieu chua prior

        for idx, c in enumerate(self._classes):
            X_c = X[y == c] #mean each class, i_row and each(all) column
            self._mean[idx, :] = X_c.mean(axis=0)
            self._var[idx, :] = X_c.var(axis=0)#var each class, i_row and each(all) column
            self._priors[idx] = X_c.shape[0] / float(n_samples)#prior this class, X_i.shape[0]: number of this class, diveded with number of total samples

    def predict(self, X):#def predict doan label tu mau test
        y_pred = [self._predict(x) for x in X]
        
        return np.array(y_pred)

    def _predict(self, x):#apply function y = argmaxXi Log(P(xi/y)) +... Log(P(y))
        posteriors = []

        # calculate posterior probability for each class
        for idx, c in enumerate(self._classes):
            prior = np.log(self._priors[idx])
            posterior = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)

        # return class with highest posterior probability
        return self._classes[np.argmax(posteriors)] #tra ve label duoc phan loai

    def _pdf(self, class_idx, x):#apply formula P(xi/y) = 1/sqrt(2pi*var) * exp(- (xi-y)**2/ 2var)
        mean = self._mean[class_idx]
        var = self._var[class_idx]
        numerator = np.exp(-((x - mean) ** 2) / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator


In [15]:
#scikit-learn datasets make_classification
X, y = datasets.make_classification(n_samples = 1000, n_features = 10, n_classes = 2, random_state = 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 70)

nb = NaiveBayes()
nb.fit(X_train, y_train)
pre = nb.predict(X_test)

print(accuracy_score(y_test, pre))

0.9366666666666666


In [16]:
#scikit-learn datasets Iris
df = datasets.load_iris()
A= df.data
b = df.target
A_train, A_test, b_train, b_test = train_test_split(A,b, test_size = 0.25, random_state = 40)

nb2 = NaiveBayes()
nb2.fit(A_train, b_train)
pre2 = nb2.predict(A_test)

print(accuracy_score(b_test, pre2))
#print(A_test, pre2)

1.0


In [19]:
'''Ví dụ thứ 2, sử dụng dataset pandas .csv, sử dụng Multinomial Classifier của thư viện sklearn.naive_bayes với mục tiêu phân loại
đâu là email spam = spam và đâu là email not spam = ham '''
import pandas as pd
from  sklearn.naive_bayes import MultinomialNB as NB
#them class 0 1 cho data
dataset = pd.read_csv('spam.csv')
dataset['Spam'] = dataset['Category'].apply(lambda x: 1 if x == 'spam' else 0)
print(dataset.head())

#chia class va features
X1 = dataset['Message']
y1 = dataset['Spam']
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.2)

#dua text trong X ve du lieu number
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer()
X_train_count = v.fit_transform(X1_train)
X_train_count.toarray()[:2]

  Category                                            Message  Spam
0      ham  Go until jurong point, crazy.. Available only ...     0
1      ham                      Ok lar... Joking wif u oni...     0
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...     1
3      ham  U dun say so early hor... U c already then say...     0
4      ham  Nah I don't think he goes to usf, he lives aro...     0


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [20]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train_count,y1_train)
#test with 2 feature
emails = [
    'Hey mohan, can we get together to watch footbal game tomorrow?',
    'Upto 20% discount on parking, exclusive offer just for you. Dont miss this reward!'
]
emails_count = v.transform(emails)
model.predict(emails_count)

#evalu accuracy
X_test_count = v.transform(X1_test)
model.score(X_test_count, y1_test)

0.9838565022421525